### `Text Classification`

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100000)
from IPython.display import display # color
import os
import eli5 # weigts

# sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC # faster # vs SVC kernal 
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [4]:
# a helper for displaying the DataFrame
def highlight_col(x, df):
    # set by condition
    pos_mask = (df['label'] == 'pos')
    neg_mask = (df['label'] == 'neg')
    x = pd.DataFrame('', index=df.index, columns=df.columns)
    x.loc[pos_mask] = 'background-color: #e6ffe6'
    x.loc[neg_mask] = 'background-color: #ffe6e6'
    return x    

* `Load the data` [Here](https://www.kaggle.com/datasets/mksaad/arabic-sentiment-twitter-corpus)

In [5]:
# read tsv files for train
TRAIN_POS_PATH = os.path.join(os.getcwd(), 'Data', 'train_Arabic_tweets_positive_20190413.tsv')
TRAIN_NEG_PATH = os.path.join(os.getcwd(), 'Data', 'train_Arabic_tweets_negative_20190413.tsv')

df_train_pos = pd.read_csv(TRAIN_POS_PATH, sep='\t', header=None)
df_train_neg = pd.read_csv(TRAIN_NEG_PATH, sep='\t', header=None)

# Concate both
df_train = pd.concat([df_train_pos, df_train_neg], ignore_index=True)
df_train.columns = ['label', 'tweet']

df_train.head()

,label,tweet
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لله، لا تبحثوا فينا عن قوة، إننا مكسورون، القوة التي…
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال في روحك أماالمنبهرون بالمظا…
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترضى بغير القمه مجرد ساعات لاستعادة الصداره💛💙 الوصول إلى القمه مهارة ت…
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المسيار ..! . ترى كانوا يشجعون ريال مدريد ضد النصر 🤣


In [6]:
# No need more for them after concatenating
del df_train_pos, df_train_neg

In [7]:
# See Highlited DF with my custom function
df_tmp = df_train.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
8252,pos,يقول عز وجل ( لقد خلقنا الإنسان في أحسن تقويم ) ويقول ( و لقد كرمنا بني آدم ) لايجوز بعض الصفات التي لا تليق على ال…
15023,pos,جون جون 😍
31125,neg,فيديو مؤثر 💔 كي لاننسى ،، ولن ننسى ولن نغفر ،، نريد القصاص ، ماعايذين تخازل في اهم اللحظات الحاسمه ،، دمهم غالي ،…
21644,pos,عندما تحب أحدهم لا تخبره كثيرا عن ذلك الحب إجعله يراه .. 🍁 #تايه 💔 #اجمل_احساس_للدعم
40362,neg,💚🍃🌷📿 🥀 🥀 من السهل أن تضع يدك على فمك كي لآ. تتكلم ولكن من آلصعب أن تضع يدك على قلبك ك…


In [8]:
# read tsv files for train
TEST_POS_PATH = os.path.join(os.getcwd(), 'Data', 'test_Arabic_tweets_positive_20190413.tsv')
TEST_NEG_PATH = os.path.join(os.getcwd(), 'Data', 'test_Arabic_tweets_negative_20190413.tsv')

df_test_pos = pd.read_csv(TEST_POS_PATH, sep='\t', header=None)
df_test_neg = pd.read_csv(TEST_NEG_PATH, sep='\t', header=None)

# Concate both
df_test = pd.concat([df_test_pos, df_test_neg], ignore_index=True)
df_test.columns = ['label', 'tweet']

df_test.head()

,label,tweet
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران كان بيسلم المباراة بعد تبديل كارييو بإنتظار الإتحاد بكرة يارب يار…
1,pos,صباحك خيرات ومسرات 🌸
2,pos,"#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫 ✍ "" ~ومع ذلك هتف بالدعاء ﴿وارزقهم من الثمرات ﴾ مهماكانت ظرو…"
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويتر وربنا 😂مش اسلوب كل يومين يدخلي واحد قد جدي علشان يشقطني 😒 😹و عند…
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع امورنا واكتب لنا الفردوس نحن ووالدينا وجميع موتى المسلمين برحمتك يا ارحم الراحمين


In [9]:
# No need more for them after concatenating
del df_test_pos, df_test_neg

In [11]:
# See Highlited DF with my custom function
df_tmp = df_test.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
5586,pos,حسابات ملگية♛ 🌟🌟 مغردون مميزون.. لهم حضور أنيق وحروف رآقية 🌸 #حسابات_ملكية_تستحق_المتابعة …
5164,pos,اسرع تحويلل وصلني من #ريهام_الخير_والعطاء الله يسعدها ويعوضها اضعافها 🖤 وكل متفاعل له نصيب معها ما يضيع حق…
5643,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المسيار ..! . ترى كانوا يشجعون ريال مدريد ضد النصر 🤣
10570,neg,وجهة نظر 😂😂 أنت شككت في (هدف ) و (طرد ) المختصين استندوا على مواد قانونية وليست حالات تقديرية فعلى ماذا استندت يا…
3804,pos,❤تصبحوا على خير ❤


----

* `Baseline model (using pipeline)`

In [12]:
# Vectorizing and then model
vect = CountVectorizer(max_features=15000, encoding='utf-8') # only X ### encoding
clf = LogisticRegression(max_iter=10000) # may target be text # X and y

# combine them to a pipeline
pipe = Pipeline(steps=[
        ('vectorizer', vect),
        ('classifier', clf)
    ])

# fitting to train data # x and y for pipe
pipe.fit(df_train['tweet'], df_train['label'])

Pipeline(steps=[('vectorizer', CountVectorizer(max_features=15000)),
                ('classifier', LogisticRegression(max_iter=10000))])

In [13]:
df_train['label'].value_counts()   # balanced dataset --> acc

pos    22761
neg    22514
Name: label, dtype: int64

* `Test the Baseline`

In [14]:
# predict on test
y_pred_test = pipe.predict(df_test['tweet'])
print(f'Accuracy is {accuracy_score(df_test["label"], y_pred_test) * 100:.3f} %')

Accuracy is 77.231 %


* `Let's take a look inside the model`

In [12]:
eli5.show_weights(clf, vec=vect, top=20, feature_names=vect.get_feature_names_out())

Weight?,Feature
+2.869,الإخونج
+2.569,برونو
+2.445,وصباحك
+2.385,هالسنه
+2.057,الزرقاء
+1.996,ابريل
+1.977,اللوك
+1.970,تعبر
+1.966,حكمة
+1.904,فيديو_الهلال


* `Try our model on some tweets`

In [13]:
for _, row in df_test.sample(5).iterrows():
    print(f"True label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vect, top=10, feature_names=vect.get_feature_names_out()))
    print("--"*50)

# Green: Features that support the predicted class. These features increase the likelihood of the model predicting the shown class.
# Red: Features that support the opposite class. These features decrease the likelihood of the model predicting the shown class.

True label: pos


----------------------------------------------------------------------------------------------------
True label: pos


----------------------------------------------------------------------------------------------------
True label: pos


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------


----

* `Try Tfidf with some Processing`

In [14]:
# Using TFIDF and SVC in one pipeline
# char_wb: will generate character n-grams within each word separately and will not span across the boundaries of different word
vect = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=0.01, max_df=0.5, max_features=10000)
clf = LinearSVC() # may target text
pipe_tfidf = Pipeline(steps=[
        ('vectorizer', vect),
        ('classifier', clf)
    ])
pipe_tfidf.fit(df_train['tweet'], df_train['label'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.5,
                                 max_features=10000, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('classifier', LinearSVC())])

In [15]:
# predict on test
y_pred_test = pipe_tfidf.predict(df_test['tweet'])
print(f'Accuracy is {accuracy_score(df_test["label"], y_pred_test) * 100:.3f} %')

Accuracy is 83.811 %


In [16]:
for _, row in df_test.sample(5).iterrows():
    print(f"True label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vect, top=10, feature_names=vect.get_feature_names_out()))
    print("--"*50)

True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: pos


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------


* `Feel free to play with notebook explore different models with different datasets`

In [17]:
import joblib
joblib.dump(pipe_tfidf, 'model.pkl')

['model.pkl']

---